Table 3. Layer detail for multiscale-CNN/LSTM (ms-C/L)

also supports Table 5. Model and results summary for Opportunity dataset

In [2]:
%pylab inline
import pickle
import glob
import os
import pandas as pd
import numpy as np
import sys

sys.path.insert(0, ".")

import seaborn as sns

sns.set()
sns.set_style("whitegrid")
sns.set_context("paper")


def pd_full(rows=100):
    pd.set_option("display.max_rows", rows)
    pd.set_option("display.max_columns", None)
    pd.set_option("display.width", 2000)
    pd.set_option("display.float_format", "{:20,.3f}".format)
    pd.set_option("display.max_colwidth", None)


pd_full()

import torch
import wardmetrics

torch.__version__, wardmetrics.__version__

Populating the interactive namespace from numpy and matplotlib


('1.0.1.post2', '0.9.5')

In [3]:
# Load relevant experiment script
import scripts.run_ensemble_exp as exp
from filternet.training.evalmodel import *
from filternet.training.ensemble_train import *
from filternet.models.reference_architectures import ref_archs

In [5]:
# ref_arch = 'base_cnn'
# print('*' * 40)
# print(ref_arch)

# config = {}
# config["base_config"] = ref_arch
# # config["model_config"] = {} #get_ref_arch("multi_scale_cnn_lstm")
# # config["name"] = f"{NAME}_{ref_arch}_{i}"

# trainer = Trainer(**config)
# trainer.cuda = False
# trainer.init_data()

# from torchsummaryX import summary

# df_base_cnn = summary(
#     trainer.model,
#     torch.zeros(
#         (trainer.batch_size, trainer.data_spec["input_channels"], trainer.win_len,),
#         device=trainer.device,
#     ),
# )

# print(ref_arch)
# print('*' * 40)

In [5]:
# # deep conv lstm
# em = load_eval_model_from_dir('saved_models/base_configs_5_deepconvlstm_0')
# ref_arch = 'deepconvlstm'
# print('*' * 40)
# print(ref_arch)
# trainer = em.trainer


# from torchsummaryX import summary
# trainer.model.cuda()

# df_multi_scale_cnn_lstm = summary(
#     trainer.model,
#     torch.zeros(
#         (100, trainer.data_spec["input_channels"], 24,),
#         device=trainer.device,
#     ),
# )

# # print(ref_arch)
# # print('*' * 40)

```
****************************************
deepconvlstm
=======================================================================================================
                             Kernel Shape        Output Shape     Params  Mult-Adds
Layer                                                                              
0_conv_stack.Conv2d_0      [1, 64, 5, 1]   [100, 64, 20, 113]  384.0      723.2k   
1_conv_stack.ReLU_1        -               [100, 64, 20, 113]  -          -        
2_conv_stack.Conv2d_2      [64, 64, 5, 1]  [100, 64, 16, 113]  20.544k    37.02784M
3_conv_stack.ReLU_3        -               [100, 64, 16, 113]  -          -        
4_conv_stack.Conv2d_4      [64, 64, 5, 1]  [100, 64, 12, 113]  20.544k    27.77088M
5_conv_stack.ReLU_5        -               [100, 64, 12, 113]  -          -        
6_conv_stack.Conv2d_6      [64, 64, 5, 1]  [100, 64, 8, 113]   20.544k    18.51392M
7_conv_stack.ReLU_7        -               [100, 64, 8, 113]   -          -        
8_drop1                    -               [8, 100, 7232]      -          -        
9_lstm1                    -               [8, 100, 128]       3.769344M  3.76832M 
10_drop2                   -               [8, 100, 128]       -          -        
11_lstm2                   -               [8, 100, 128]       132.096k   131.072k 
12_end_stacks.0.Dropout_0  -               [100, 128, 8]       -          -        
13_end_stacks.0.Conv1d_1   [128, 18, 1]    [100, 18, 8]        2.322k     18.432k  
-------------------------------------------------------------------------------------------------------
                          Totals
Total params          3.965778M 
Trainable params      3.965778M 
Non-trainable params  0.0       
Mult-Adds             87.953664M
```

In [7]:
ref_arch = "multi_scale_cnn_lstm"
print("*" * 40)
print(ref_arch)

config = {}
config["base_config"] = ref_arch
# config["model_config"] = {} #get_ref_arch("multi_scale_cnn_lstm")
# config["name"] = f"{NAME}_{ref_arch}_{i}"

trainer = Trainer(**config)
trainer.cuda = False
trainer.init_data()

from torchsummaryX import summary

df_multi_scale_cnn_lstm = summary(
    trainer.model,
    torch.zeros(
        (trainer.batch_size, trainer.data_spec["input_channels"], trainer.win_len,),
        device=trainer.device,
    ),
)

print(ref_arch)
print("*" * 40)

****************************************
multi_scale_cnn_lstm
Loading cached data.
Loaded.
Batch size: 9
                                  Kernel Shape   Output Shape  Params Mult-Adds
Layer                                                                          
0_down_stack_1.0.Dropout2d_0     -              [9, 113, 512]  -       -       
1_down_stack_1.0.Conv1d_1        [113, 100, 5]  [9, 100, 512]  56.6k   28.928M 
2_down_stack_1.0.ReLU_2          -              [9, 100, 512]  -       -       
3_down_stack_1.0.BatchNorm1d_3   [100]          [9, 100, 512]  200.0   100.0   
4_down_stack_1.1.Dropout2d_0     -              [9, 100, 512]  -       -       
5_down_stack_1.1.Conv1d_1        [100, 100, 5]  [9, 100, 512]  50.1k   25.6M   
6_down_stack_1.1.ReLU_2          -              [9, 100, 512]  -       -       
7_down_stack_1.1.AvgPool1d_3     -              [9, 100, 256]  -       -       
8_down_stack_1.1.BatchNorm1d_4   [100]          [9, 100, 256]  200.0   100.0   
9_down_stack_1.

In [6]:
def flm_layer_arch(
    input_size,
    output_size,
    kernel_size=5,
    type="cnn",
    stride=1,
    pool=None,
    dropout=0.1,
    stride_pos=None,
    batch_norm=True,
    groups=1,
):
    """Calculate architecture properties of a FLM layer"""

    s = pd.Series()
    s["input_size"] = input_size
    s["output_size"] = output_size
    s["type"] = type
    m = s["input_size"]
    n = s["output_size"]
    if s["type"] == "cnn":
        s["kernel_size"] = kernel_size
        s["params"] = m * n * s["kernel_size"] + n
    if s["type"] == "lstm":
        m = s["input_size"]
        n = s["output_size"]
        s["params"] = 4 * (n * m + n ** 2 + n)

    s["stride"] = stride
    s["batch_norm"] = batch_norm

    if dropout:
        s["dropout"] = dropout
    if batch_norm:
        s["params"] += s["output_size"]

    return s

In [7]:
###
# This code follows the same logic we use to construct FilterNet models, but instead
#  of making one, it just calculates how big the layers would be, etc.

from models.filter_net import DEFAULT_WIDTH


def describe_filternet_layers(
    n_pre=1,
    w_pre=DEFAULT_WIDTH,
    n_strided=3,
    w_strided=DEFAULT_WIDTH,
    n_interp=4,
    w_interp=DEFAULT_WIDTH,
    n_dense_pre_l=1,
    w_dense_pre_l=DEFAULT_WIDTH,
    n_l=1,
    w_l=DEFAULT_WIDTH,
    n_dense_post_l=0,
    w_dense_post_l=int(DEFAULT_WIDTH / 2),
    cnn_kernel_size=5,
    scale=1.0,
    bn_pre=False,
    dropout=0.1,
    do_pool=True,
    stride_pos="post",
    stride_amt=2,
    num_output_classes=18,
    **kwargs
):

    w_pre = int((w_pre * scale))  # / 6) * 6
    w_strided = int((w_strided * scale))  # / 6) * 6
    w_interp = int(w_interp * scale)
    w_dense_pre_l = int(w_dense_pre_l * scale)
    w_l = int((w_l * scale) / 2) * 2
    w_dense_post_l = int(w_dense_post_l * scale)

    df = pd.DataFrame()

    s = pd.Series()
    s["input_size"] = 0
    s["output_size"] = 113
    s["type"] = "input"
    s["params"] = 0
    s["stride"] = 1
    s["stride_ratio"] = 1
    s["batch_norm"] = False
    s["cone_size"] = 1
    s["component"] = ""
    df = df.append(s, ignore_index=True)
    in_shape = s.output_size

    for i in range(n_pre):
        s = flm_layer_arch(
            in_shape, w_pre, cnn_kernel_size, type="cnn", dropout=dropout
        )
        s["stride_ratio"] = df.stride_ratio.iloc[-1] * s["stride"]
        s["cone_size"] = (
            df.cone_size.iloc[-1]
            + s["stride"] * (s["kernel_size"] - 1) * df.stride_ratio.iloc[-1]
        )
        s["component"] = "A"
        df = df.append(s, ignore_index=True)
        in_shape = s.output_size

    for i in range(n_strided):
        stride = stride_amt
        pool = stride if (do_pool and stride > 1) else None
        ltype = "cnn"
        s = flm_layer_arch(
            in_shape,
            w_strided,
            cnn_kernel_size,
            type=ltype,
            stride=stride,
            pool=pool,
            stride_pos=stride_pos,
            dropout=dropout,
            # groups=3 if ( i % 2 == 0 ) else 2
        )
        s["stride_ratio"] = df.stride_ratio.iloc[-1] * s["stride"]
        s["cone_size"] = (
            df.cone_size.iloc[-1]
            + s["stride"] * (s["kernel_size"] - 1) * df.stride_ratio.iloc[-1]
        )
        s["component"] = "B"
        #     self.output_stride *= stride
        df = df.append(s, ignore_index=True)
        in_shape = s.output_size

    ds_1_end_size = in_shape

    ds2_ltype = "cnn"
    down_stack_2 = []
    output_stride_ratio = s["stride_ratio"]
    merged_output_size = in_shape

    for i in range(n_interp):
        stride = stride_amt if (i < n_interp - 1) else 1
        pool = stride if (do_pool and stride > 1) else None
        w = int(np.ceil(w_interp * 0.5 ** (i + 1)))
        # if i == n_interp-1:
        #     w = int(w_interp * .66)
        # if i == n_interp - 2:
        #     w =int(w_interp * .33)
        # else:
        #     w = w_interp

        s = flm_layer_arch(
            in_shape,
            w,
            cnn_kernel_size,
            type=ds2_ltype,
            stride=stride,
            pool=pool,
            stride_pos=stride_pos,
            dropout=dropout,
            # groups=3 if ( i % 2 == 0 ) else 2
        )
        s["stride_ratio"] = df.stride_ratio.iloc[-1] * s["stride"]
        s["cone_size"] = (
            df.cone_size.iloc[-1]
            + s["stride"] * (s["kernel_size"] - 1) * df.stride_ratio.iloc[-1]
        )
        s["component"] = "C"
        #     self.output_stride *= stride
        df = df.append(s, ignore_index=True)
        in_shape = s.output_size

        merged_output_size += in_shape

    in_shape = merged_output_size

    s = pd.Series()
    s["input_size"] = merged_output_size
    s["output_size"] = merged_output_size
    s["type"] = "concat/interp"
    s["params"] = 0
    s["stride_ratio"] = output_stride_ratio
    s["cone_size"] = df.cone_size.max()
    s["batch_norm"] = False
    s["component"] = "D"
    df = df.append(s, ignore_index=True)
    in_shape = s.output_size

    for i in range(n_dense_pre_l):
        s = flm_layer_arch(
            in_shape, w_dense_pre_l, kernel_size=1, type="cnn", dropout=dropout
        )
        s["stride_ratio"] = df.stride_ratio.iloc[-1] * s["stride"]
        s["cone_size"] = (
            df.cone_size.iloc[-1]
            + s["stride"] * (s["kernel_size"] - 1) * df.stride_ratio.iloc[-1]
        )
        s["component"] = "E"
        #     self.output_stride *= stride
        df = df.append(s, ignore_index=True)
        in_shape = s.output_size

    for i in range(n_l):
        s = flm_layer_arch(
            in_shape,
            w_l,
            cnn_kernel_size,  # unused when type!-='cnn'
            type="lstm",
            dropout=dropout,
        )
        s["stride_ratio"] = output_stride_ratio
        s["cone_size"] = np.nan
        s["component"] = "F"
        df = df.append(s, ignore_index=True)
        in_shape = s.output_size

    for i in range(n_dense_post_l):
        s = flm_layer_arch(
            in_shape, w_dense_post_l, kernel_size=1, type="cnn", dropout=dropout
        )
        s["stride_ratio"] = output_stride_ratio
        s["cone_size"] = (
            df.cone_size.iloc[-1]
            + s["stride"] * (s["kernel_size"] - 1) * df.stride_ratio.iloc[-1]
        )  #     self.output_stride *= stride
        df = df.append(s, ignore_index=True)
        in_shape = s.output_size

    # end stack
    s = flm_layer_arch(
        in_shape,
        num_output_classes,
        kernel_size=1,
        type="cnn",
        dropout=dropout,
        batch_norm=False,
    )
    s["stride_ratio"] = output_stride_ratio
    s["cone_size"] = (
        df.cone_size.iloc[-1]
        + s["stride"] * (s["kernel_size"] - 1) * df.stride_ratio.iloc[-1]
    )
    s["component"] = "G"
    s["type"] = "cnn/output"
    df = df.append(s, ignore_index=True)

    for c in [
        "output_size",
        "input_size",
        "kernel_size",
        "params",
        "stride",
        "stride_ratio",
        "cone_size",
    ]:
        df[c] = df[c].fillna(0).astype(int)

    for c in ["batch_norm"]:
        df[c] = df[c].fillna(0).astype(bool)

    # for c in ['output_size', 'batch_norm', 'dropout', 'input_size', 'kernel_size', 'params', 'stride', 'type']
    df = df[
        [
            "component",
            "type",
            "input_size",
            "output_size",
            "stride",
            "kernel_size",
            "dropout",
            "batch_norm",
            "params",
            "stride_ratio",
            "cone_size",
        ]
    ]

    return df

In [9]:
###
# Now we go through and create a description using each of our reference architectures.
dd = {}

In [10]:
name = "base_cnn"
df = describe_filternet_layers(**get_ref_arch("base_cnn"))
d = {
    "output_stride_ratio": df.stride_ratio.iloc[-1],
    "cone_size": df.cone_size.iloc[-1],
    "modules": len(df) - 1,
    "trainable_params": df.params.sum(),
}
print(name)
print(d)
dd[name] = d
df

base_cnn
{'output_stride_ratio': 8, 'cone_size': 61, 'modules': 6, 'trainable_params': 209118}


/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:134: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,component,type,input_size,output_size,stride,kernel_size,dropout,batch_norm,params,stride_ratio,cone_size
0,,input,0,113,1,0,nan,False,0,1,1
1,A,cnn,113,100,1,5,0.100,True,56700,1,5
2,B,cnn,100,100,2,5,0.100,True,50200,2,13
3,B,cnn,100,100,2,5,0.100,True,50200,4,29
4,B,cnn,100,100,2,5,0.100,True,50200,8,61
5,D,concat/interp,100,100,0,0,nan,False,0,8,61
6,G,cnn/output,100,18,1,1,0.100,False,1818,8,61


In [11]:
describe_filternet_layers()
name = "multi_scale_cnn"
df = describe_filternet_layers(**get_ref_arch("multi_scale_cnn"))
d = {
    "output_stride_ratio": df.stride_ratio.iloc[-1],
    "cone_size": df.cone_size.iloc[-1],
    "modules": len(df) - 1,
    "trainable_params": df.params.sum(),
}
print(name)
print(d)
dd[name] = d
df

/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:134: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


multi_scale_cnn
{'output_stride_ratio': 8, 'cone_size': 765, 'modules': 11, 'trainable_params': 262338}


,component,type,input_size,output_size,stride,kernel_size,dropout,batch_norm,params,stride_ratio,cone_size
0,,input,0,113,1,0,nan,False,0,1,1
1,A,cnn,113,100,1,5,0.100,True,56700,1,5
2,B,cnn,100,100,2,5,0.100,True,50200,2,13
3,B,cnn,100,100,2,5,0.100,True,50200,4,29
4,B,cnn,100,100,2,5,0.100,True,50200,8,61
5,C,cnn,100,50,2,5,0.100,True,25100,16,125
6,C,cnn,50,25,2,5,0.100,True,6300,32,253
7,C,cnn,25,13,2,5,0.100,True,1651,64,509
8,C,cnn,13,7,1,5,0.100,True,469,64,765
9,D,concat/interp,195,195,0,0,nan,False,0,8,765


In [12]:
name = "cnn_lstm"
df = describe_filternet_layers(**get_ref_arch("cnn_lstm"))
d = {
    "output_stride_ratio": df.stride_ratio.iloc[-1],
    "cone_size": df.cone_size.iloc[-1],
    "modules": len(df) - 1,
    "trainable_params": df.params.sum(),
}
print(name)
print(d)
dd[name] = d
df

cnn_lstm
{'output_stride_ratio': 8, 'cone_size': 0, 'modules': 8, 'trainable_params': 299818}


/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:134: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,component,type,input_size,output_size,stride,kernel_size,dropout,batch_norm,params,stride_ratio,cone_size
0,,input,0,113,1,0,nan,False,0,1,1
1,A,cnn,113,100,1,5,0.100,True,56700,1,5
2,B,cnn,100,100,2,5,0.100,True,50200,2,13
3,B,cnn,100,100,2,5,0.100,True,50200,4,29
4,B,cnn,100,100,2,5,0.100,True,50200,8,61
5,D,concat/interp,100,100,0,0,nan,False,0,8,61
6,E,cnn,100,100,1,1,0.100,True,10200,8,61
7,F,lstm,100,100,1,0,0.100,True,80500,8,0
8,G,cnn/output,100,18,1,1,0.100,False,1818,8,0


In [13]:
name = "multi_scale_cnn_lstm"
df = describe_filternet_layers(**get_ref_arch("multi_scale_cnn_lstm"))
d = {
    "output_stride_ratio": df.stride_ratio.iloc[-1],
    "cone_size": df.cone_size.iloc[-1],
    "modules": len(df) - 1,
    "trainable_params": df.params.sum(),
}
print(name)
print(d)
dd[name] = d
df

multi_scale_cnn_lstm
{'output_stride_ratio': 8, 'cone_size': 0, 'modules': 12, 'trainable_params': 342838}


/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:134: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,component,type,input_size,output_size,stride,kernel_size,dropout,batch_norm,params,stride_ratio,cone_size
0,,input,0,113,1,0,nan,False,0,1,1
1,A,cnn,113,100,1,5,0.100,True,56700,1,5
2,B,cnn,100,100,2,5,0.100,True,50200,2,13
3,B,cnn,100,100,2,5,0.100,True,50200,4,29
4,B,cnn,100,100,2,5,0.100,True,50200,8,61
5,C,cnn,100,50,2,5,0.100,True,25100,16,125
6,C,cnn,50,25,2,5,0.100,True,6300,32,253
7,C,cnn,25,13,2,5,0.100,True,1651,64,509
8,C,cnn,13,7,1,5,0.100,True,469,64,765
9,D,concat/interp,195,195,0,0,nan,False,0,8,765


In [14]:
name = "4-fold multi_scale_cnn_lstm Ensemble"
df = describe_filternet_layers(**get_ref_arch("multi_scale_cnn_lstm"))
d = {
    "output_stride_ratio": df.stride_ratio.iloc[-1],
    "cone_size": df.cone_size.iloc[-1],
    "modules": len(df) - 1,
    "trainable_params": df.params.sum() * 4,
}
print(name)
print(d)
dd[name] = d
df

4-fold multi_scale_cnn_lstm Ensemble
{'output_stride_ratio': 8, 'cone_size': 0, 'modules': 12, 'trainable_params': 1371352}


/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:134: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,component,type,input_size,output_size,stride,kernel_size,dropout,batch_norm,params,stride_ratio,cone_size
0,,input,0,113,1,0,nan,False,0,1,1
1,A,cnn,113,100,1,5,0.100,True,56700,1,5
2,B,cnn,100,100,2,5,0.100,True,50200,2,13
3,B,cnn,100,100,2,5,0.100,True,50200,4,29
4,B,cnn,100,100,2,5,0.100,True,50200,8,61
5,C,cnn,100,50,2,5,0.100,True,25100,16,125
6,C,cnn,50,25,2,5,0.100,True,6300,32,253
7,C,cnn,25,13,2,5,0.100,True,1651,64,509
8,C,cnn,13,7,1,5,0.100,True,469,64,765
9,D,concat/interp,195,195,0,0,nan,False,0,8,765


In [15]:
name = "base_lstm"
df = describe_filternet_layers(**get_ref_arch("base_lstm"))
d = {
    "output_stride_ratio": df.stride_ratio.iloc[-1],
    "cone_size": df.cone_size.iloc[-1],
    "modules": len(df) - 1,
    "trainable_params": df.params.sum(),
}
print(name)
print(d)
dd[name] = d
df

base_lstm
{'output_stride_ratio': 1, 'cone_size': 0, 'modules': 3, 'trainable_params': 87518}


/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:134: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app


,component,type,input_size,output_size,stride,kernel_size,dropout,batch_norm,params,stride_ratio,cone_size
0,,input,0,113,1,0,nan,False,0,1,1
1,D,concat/interp,113,113,0,0,nan,False,0,1,1
2,F,lstm,113,100,1,0,0.100,True,85700,1,0
3,G,cnn/output,100,18,1,1,0.100,False,1818,1,0


In [16]:
name = "multi_scale_cnn_lstm_0.50"
df = describe_filternet_layers(scale=0.5, **get_ref_arch("multi_scale_cnn_lstm"))
d = {
    "output_stride_ratio": df.stride_ratio.iloc[-1],
    "cone_size": df.cone_size.iloc[-1],
    "modules": len(df) - 1,
    "trainable_params": df.params.sum(),
}
print(name)
print(d)
dd[name] = d
df

multi_scale_cnn_lstm_0.50
{'output_stride_ratio': 8, 'cone_size': 0, 'modules': 12, 'trainable_params': 100936}


/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:134: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,component,type,input_size,output_size,stride,kernel_size,dropout,batch_norm,params,stride_ratio,cone_size
0,,input,0,113,1,0,nan,False,0,1,1
1,A,cnn,113,50,1,5,0.100,True,28350,1,5
2,B,cnn,50,50,2,5,0.100,True,12600,2,13
3,B,cnn,50,50,2,5,0.100,True,12600,4,29
4,B,cnn,50,50,2,5,0.100,True,12600,8,61
5,C,cnn,50,25,2,5,0.100,True,6300,16,125
6,C,cnn,25,13,2,5,0.100,True,1651,32,253
7,C,cnn,13,7,2,5,0.100,True,469,64,509
8,C,cnn,7,4,1,5,0.100,True,148,64,765
9,D,concat/interp,99,99,0,0,nan,False,0,8,765


In [17]:
name = "multi_scale_cnn_lstm_2.00"
df = describe_filternet_layers(scale=2.0, **get_ref_arch("multi_scale_cnn_lstm"))
d = {
    "output_stride_ratio": df.stride_ratio.iloc[-1],
    "cone_size": df.cone_size.iloc[-1],
    "modules": len(df) - 1,
    "trainable_params": df.params.sum(),
}
print(name)
print(d)
dd[name] = d
df

multi_scale_cnn_lstm_2.00
{'output_stride_ratio': 8, 'cone_size': 0, 'modules': 12, 'trainable_params': 1250469}


/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda3/envs/petinsight/lib/python3.7/site-packages/ipykernel_launcher.py:134: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,component,type,input_size,output_size,stride,kernel_size,dropout,batch_norm,params,stride_ratio,cone_size
0,,input,0,113,1,0,nan,False,0,1,1
1,A,cnn,113,200,1,5,0.100,True,113400,1,5
2,B,cnn,200,200,2,5,0.100,True,200400,2,13
3,B,cnn,200,200,2,5,0.100,True,200400,4,29
4,B,cnn,200,200,2,5,0.100,True,200400,8,61
5,C,cnn,200,100,2,5,0.100,True,100200,16,125
6,C,cnn,100,50,2,5,0.100,True,25100,32,253
7,C,cnn,50,25,2,5,0.100,True,6300,64,509
8,C,cnn,25,13,1,5,0.100,True,1651,64,765
9,D,concat/interp,388,388,0,0,nan,False,0,8,765


In [27]:
###
# taken from DeepConvLSTM paper or manual calculations
name = "deepconvlstm"
d = {
    "output_stride_ratio": 12,  # sliding window step
    "cone_size": 24,  # win_len
    "modules": 0,
    "trainable_params": int(3.965778e6),
}
print(name)
print(d)
dd[name] = d
# df

deepconvlstm
{'output_stride_ratio': 12, 'cone_size': 24, 'modules': 0, 'trainable_params': 3965778}


In [22]:
sorted(dd.keys())

['4-fold multi_scale_cnn_lstm Ensemble',
 'base_cnn',
 'base_lstm',
 'cnn_lstm',
 'deepconvlstm',
 'multi_scale_cnn',
 'multi_scale_cnn_lstm',
 'multi_scale_cnn_lstm_0.50',
 'multi_scale_cnn_lstm_2.00']

In [24]:
# Architecture summary
df_archs = pd.DataFrame(dd).T
df_archs

,output_stride_ratio,cone_size,modules,trainable_params
base_cnn,8,61,6,209118
multi_scale_cnn,8,765,11,262338
cnn_lstm,8,0,8,299818
multi_scale_cnn_lstm,8,0,12,342838
4-fold multi_scale_cnn_lstm Ensemble,8,0,12,1371352
base_lstm,1,0,3,87518
multi_scale_cnn_lstm_0.50,8,0,12,100936
multi_scale_cnn_lstm_2.00,8,0,12,1250469
deepconvlstm,12,24,0,3965778


In [26]:
## Output as dict
#
## COPY THIS into any notebook that needs it to make, e.g., a combined table
#
df_archs.to_dict()

{'output_stride_ratio': {'base_cnn': 8,
  'multi_scale_cnn': 8,
  'cnn_lstm': 8,
  'multi_scale_cnn_lstm': 8,
  '4-fold multi_scale_cnn_lstm Ensemble': 8,
  'base_lstm': 1,
  'multi_scale_cnn_lstm_0.50': 8,
  'multi_scale_cnn_lstm_2.00': 8,
  'deepconvlstm': 12},
 'cone_size': {'base_cnn': 61,
  'multi_scale_cnn': 765,
  'cnn_lstm': 0,
  'multi_scale_cnn_lstm': 0,
  '4-fold multi_scale_cnn_lstm Ensemble': 0,
  'base_lstm': 0,
  'multi_scale_cnn_lstm_0.50': 0,
  'multi_scale_cnn_lstm_2.00': 0,
  'deepconvlstm': 24},
 'modules': {'base_cnn': 6,
  'multi_scale_cnn': 11,
  'cnn_lstm': 8,
  'multi_scale_cnn_lstm': 12,
  '4-fold multi_scale_cnn_lstm Ensemble': 12,
  'base_lstm': 3,
  'multi_scale_cnn_lstm_0.50': 12,
  'multi_scale_cnn_lstm_2.00': 12,
  'deepconvlstm': 0},
 'trainable_params': {'base_cnn': 209118,
  'multi_scale_cnn': 262338,
  'cnn_lstm': 299818,
  'multi_scale_cnn_lstm': 342838,
  '4-fold multi_scale_cnn_lstm Ensemble': 1371352,
  'base_lstm': 87518,
  'multi_scale_cnn_lstm